
Date: 25/08/2019

Version: 1.0

Environment: Python 2.7.11 and Jupyter notebook

### The given task is to perform parsing data such as Analyzing Sample input and Output file, extracting data of the columns, removing all the html tags and converting it into csv and json format.

<br>

# Section 1.  Import libraries for performing various tasks using in-built functions of these libraries
- importing pandas library to utilise it's method to generate the csv file
- importing regex library to utilise it's methods to analyse and assess the given data


In [1]:
import pandas as pd
import re

<br>

# Section 2.1. Parse Group145.txt File and Defining functions
- This section is created to read the text file and performing the operation on it line by line using readlines() method

## 2.1.1 Parsing file

In [2]:
file_name = 'Group145.txt'      # Initializing file name to variable file_name
file = open(file_name, 'r')     # Initializing file variable to open the given text file to read the data 
data = file.readlines()         # Reading each line individually from the given data 

## 2.1.2 Creating function for cleaning XML tags
- Creating a function for cleaning all the xml tags provided in the data
- Data in text variable as a string is being passed into this method

In [3]:
def clean_tags(text):
    # Initializing a string to save cleaned data after the xml tags are removed
    clean_text = ''
    
    # compiling the pattern for fetching texts between the XML tag
    clean_pattern = re.compile('<.*?>|\n')
    
    # Using substitute method for replacing xml tags with blanks in the texts 
    clean_text = re.sub(clean_pattern, '', text)
    
    # Using replace method to replace any \ (backslash) in the fetched texts
    clean_text = clean_text.replace('\\', '')
    
    # returning cleaned text to the calling method
    return clean_text

## 2.1.3 Creating function for converting list into CSV file
- Creating a function for converting list data into csv file
- Data in the conversion_data is passed as a list, converted into dataframes and converting it into csv file

In [4]:
def convert_to_csv(conversion_data):
    # Initializing variable for storing dataframe
    df = pd.DataFrame(conversion_data)
    
    # Conveting dataframe df to csv file
    df.to_csv('Group145.csv', index=False)

## 2.1.4 Creating function for converting list to JSON file

- Creating a function for converting list data into json file
- Passing all the list data into function

In [5]:
def convert_to_json(header, grant_id, patent_title, kind, number_of_claims, inventor_name, citation_applicant_count, citation_examiner_count, claim_text, abstract):
    # Initializing variables for storing data
    temp_data = []
    data = []
    json_data = []
    
    # For loop for creating json format from patent_title till abstract
    for i in range(len(grant_id)):
       temp_data.append('{"'+header[0]+'":"'+ str(patent_title[i]) +'","'+header[1]+'":"'+ str(kind[i]) +'","'+header[2]+'":"'+ str(number_of_claims[i]) +'","'+header[3]+'":"'+ str(inventor_name[i]) +'","'+header[4]+'":"'+ str(citation_applicant_count[i]) +'","'+header[5]+'":"'+ str(citation_examiner_count[i]) +'","'+header[6]+'":"'+ str(claim_text[i]) +'","'+header[7]+'":"'+ str(abstract[i]) +'"\n}')
    
    # For loop for appending Grant_id into list created above
    for j in range(len(grant_id)):
        data.append('\n"'+ grant_id[j] +'"' + ':'+ temp_data[j])
    
    # Appending start and end { } in the created json format
    json_data = '{'+",".join(data) + '}'
    
    # Returning created json format data as a list
    return json_data

<br>

# Section 2.2. Retrieving all the column data

## 2.2.1 Finding Grant

- This section fetches grant id from the given xml data and stores into grant_id list

In [6]:
# Initializing grant_id list to store the grant id being fetched from the data
grant_id = [] 

# for loop to read through each line of the data being given
for i in data:
    
    # if condition to check the given tag exist in the data
    if re.search(r'file="(.*?)-20190723.XML', i):
        
        # Storing the value of the grant id into a temporary variable by using the provided regex  
        grant_id_temp = re.search(r'file="(.*?)-20190723.XML', i)
        
        # Appending the fetched value inside the grant_id list
        grant_id.append(grant_id_temp.group(1))

## 2.2.2 Finding Patent Title

- This section fetches patent title from the given xml data and stores into patent_title list

In [7]:
# Initializing patent_title list to store the patent title being fetched from the data
patent_title = []

# for loop to read through each line of the data being given
for i in data:
    
    # if condition to check the given tag exist in the data
    if re.search(r'<invention-title id=\"\w{5,6}\">(.*?)</invention-title>', i):
        
        # Storing the value of the patent title into temporary variable by using the provided regex
        patent_title_temp = re.search(r'<invention-title id=\"\w{5,6}\">(.*?)</invention-title>', i)
        
        # Appending the fetched value inside the patent_title list
        patent_title.append(patent_title_temp.group(1))

## 2.2.3 Finding Kind
- This section fetches kind from the given xml data and stores into kind list

In [8]:
# Initializin kind list to store the kind being fetched from the data
kind = []

# Initializing all the counters to 0 for fetching the index of kind value 
counter = kind_start_counter = kind_end_counter = 0

# Using for loop to read through each line of the data being given
for i in data:
    # Incrementing counter by 1 in each iteration to get index of each line
    counter += 1
    
    # if condition to check the given tag exist in the data
    if re.search(r'<publication-reference>', i):
        
        # Assigning index of the tag to kind_start_counter
        kind_start_counter = counter
    
    # elif condition to check the given tag exist in the data
    elif re.search(r'</publication-reference>', i):
        
        # Assigning index of the tag to kind_end_counter
        kind_end_counter = counter
        
        # for loop to read between tags
        for j in data[kind_start_counter: kind_end_counter]:
            
            # if condition to check the given tag exist in the data, 
            # if the tag exist then comparing it with the specific kind type and substituting the value in the kind list
            if re.findall(r'<kind>(.*?)</kind>', j):
                value = re.search(r'<kind>(.*?)</kind>', j)
                if value.group(1) == 'B1':
                    kind.append('Utility Patent Grant (no published application) issued on or after January 2, 2001.')
                elif value.group(1) == 'B2':
                    kind.append('Utility Patent Grant (with a published application) issued on or after January 2, 2001.') 
                elif value.group(1) == 'S1':
                    kind.append('Design Patent.')
                elif value.group(1) == 'E1':
                    kind.append('Reissue Patent')
                elif value.group(1) == 'P2':
                    kind.append('Plant Patent Grant (no published application) issued on or after January 2, 2001')
                elif value.group(1) == 'P3':
                    kind.append('Plant Patent Grant (with a published application) issued on or after January 2, 2001')

## 2.2.4 Finding Number of Claims
- This section fetches Number of Claims from the given xml data and stores into number_of_claims list

In [9]:
# Initializing the number_of_claims list to store the count being fetched from the data
number_of_claims = []

# for loop to read through each line of the data being given
for i in data:
    
    # Putting if condition to check the given tag exist in the data
    if re.search(r'<number-of-claims>(.*?)<\/number-of-claims>', i):
        
        # Fetching and Storing the value of the into temporary variable by using the provided regex
        number_of_claims_temp = re.search(r'<number-of-claims>(.*?)<\/number-of-claims>', i)
        
        # Appending the fetched value inside the number_of_claims list
        number_of_claims.append(number_of_claims_temp.group(1))

## 2.2.5 Finding Inventor Name
- This section fetches Inventors name from the given xml data and stores into inventor_name list

In [10]:
# Initializing variable to save inventor name
inventor_name = []

# Initializing variable list to save temporary inventor name for each data
inventor_name_temp = []

# Initializing all the counters to 0 for fetching the index of inventor name
counter = inventor_start_counter = inventor_end_counter = 0

# for loop to read through each line of the data
for i in data:
    
    # Incrementing counter by 1 in each iteration to get index of each line
    counter += 1
    
    # if condition to check the given tag exist
    if re.search(r'<inventor sequence="001" designation="us-only">', i):
        # Assigning index to inventor_start_counter
        inventor_start_counter = counter

    # elif condition to check the given tag exist
    elif re.search(r'</inventors>', i):
        
        # Assigning index to inventor_end_counter
        inventor_end_counter = counter
        
        # for loop to read between tags
        for j in data[inventor_start_counter: inventor_end_counter]:
            
            # if condition to check the given tag exist in the data, 
            # if the tag exist then extracting into temporary variable and
            # appending it to list
            if re.findall(r'<last-name>(.*?)</last-name>', j):
                last_temp = re.search(r'<last-name>(.*?)</last-name>', j)
            if re.findall(r'<first-name>(.*?)</first-name>', j):
                first_temp = re.search(r'<first-name>(.*?)</first-name>', j)
                inventor_name_temp.append(first_temp.group(1) + ' ' + last_temp.group(1))
        inventor_name.append(inventor_name_temp)
        
        # Clearing temporary variable to store next inventor's name
        inventor_name_temp = []

## 2.2.6 Citation count for Applicant and Examiner
- This section fetches Citation count by examiner or by applicant from the given xml data

In [11]:
citation_applicant_count = []              # Initializing variable to save citation applicant count
cited_by_applicant = 'cited by applicant'  # Initializing a string to match the cited text for applicant count
applicant_counter = 0                      # Initializing a counter for number of applicant count 

citation_examiner_count = []               # Initializing variable to save citation examiner count
cited_by_examiner = 'cited by examiner'    # Initializing a string to match the cited text for examiner count
examiner_counter = 0                       # Initializing a counter for number of examiner count

# for loop to read through each line of the data
for i in data:
    
    # if condition to check the given tag exist, if exist then counting the citations
    # and counting the citation by applicant or by examinerand saving into respective variable
    if re.findall(r'<category>(.*?)</category>', i):
        cited_by = re.search(r'<category>(.*?)</category>', i)
        if cited_by.group(1) == cited_by_applicant:
            applicant_counter += 1
        elif cited_by.group(1) == cited_by_examiner:
            examiner_counter += 1
    
    # elif condition to check the end of tag for a single data, if exist then appending the count to list and 
    # resetting the temporary count to 0 for counting next data's count
    elif re.findall(r'</us-patent-grant>', i):
        citation_applicant_count.append(applicant_counter)
        citation_examiner_count.append(examiner_counter)
        applicant_counter = 0
        examiner_counter = 0

## 2.2.7 Finding Claim Text
- This section fetches Claim text from the given xml data

In [12]:
# Initializing variable to save claim text data
claim_text = []

# Initializing variable list to save temporary claim text for each data
claim_text_temp = []

# Initializing all the counters to 0 for fetching the index of Claim text
counter = claim_start_counter = claim_end_counter = 0

# for loop to read through each line of the data
for i in data:
    
    # Incrementing counter by 1 in each iteration to get index of each line
    counter += 1
    
    # if condition to check the given tag exist and assigning index value
    if re.findall(r'<claim id="CLM-00001" num="00001">', i):
        claim_start_counter = counter
        
    # elif condition to check the end of tag for a single data, if exist then assigning index value
    elif re.findall(r'</us-patent-grant>', i):
        claim_end_counter = counter
        
        # for loop from tags to fetch claim text existing between these indexes
        for j in data[claim_start_counter: claim_end_counter]:
            
            # Calling clean_tags function to get texts without HTML tags and appending it to temporary variable
            cleaned_data = clean_tags(j)
            claim_text_temp.append(cleaned_data)
            
        # Converting claim_text_temp list to string
        temp = ''.join(claim_text_temp)
        
        # Appending cleaned_text string to claim_text list
        claim_text.append(temp)
        
        # Initializing claim_text_temp and temp to null to store next data set
        claim_text_temp = []
        temp = ''

## 2.2.8 Finding Abstract
- This section fetches Abstract from the given xml data

In [13]:
# Initializing variable to save abstract data
abstract = []

# Initializing variable list to save temporary abstract for each data
abstract_data_temp = []

# Initializing counters to 0 for counting the index of each data
counter = 0

# for loop to read through each line of the data
for i in data:
    
    # Incrementing counter by 1 in each iteration to get index of each line
    counter += 1
    
    # if condition to check the given tag exist and assigning index value
    if re.findall(r'</us-bibliographic-data-grant>', i):
        abstract_start_counter = counter

    # elif condition to check the given tag exist and assigning index value
    elif re.findall(r'<drawings id="DRAWINGS">', i):
        abstract_end_counter = counter
    
    # elif condition to check the end of tag for a single data, if exist then 
    # assigning index value and looping between the above indexes
    elif re.findall(r'</us-patent-grant>', i):
        
        # for loop from tags to fetch Abstract existing between these indexes
        for j in data[abstract_start_counter: (abstract_end_counter - 1)]:
            
            # Calling clean_tags function to get texts without HTML tags and appending it to temporary variable
            cleaned_data = clean_tags(j)
            abstract_data_temp.append(cleaned_data)
            
        # Converting abstract_data_temp list to string
        temp = ''.join(abstract_data_temp)
        abstract_data_temp = []
        
        # Appending cleaned_text string to abstract list, if value is Null then appending NA else appending data
        if len(temp) == 0:
            abstract.append('NA')
        else:
            abstract.append(temp)
        temp = ''

## 2.2.9 Converting Data lists into CSV file

- This section calls convert_to_csv function for converting and writing data fetched to csv file

In [14]:
# Creating dictionary for creating csv file
dict_data = {'grant_id': grant_id, 'patent_title': patent_title, 'kind': kind, 'number_of_claims': number_of_claims, 'inventor_name': inventor_name, 'citation_applicant_count': citation_applicant_count, 'citation_examiner_count': citation_examiner_count, 'claim_text': claim_text, 'abstract': abstract}

# Calling function convert_to_csv to convert dictionary data into csv file
convert_to_csv(dict_data)

## 2.2.10 Converting Data lists into JSON file

- This section calls convert_to_json and a json formatted list is returned
- After list is returned, it is written to create output json file

In [15]:
# Header created for columns
header = ['patent_title', 'kind', 'number_of_claims', 'inventor_name', 'citation_applicant_count', 'citation_examiner_count','claim_text', 'abstract']

# Calling function convert_to_json to get json formatted list
json_data = convert_to_json(header, grant_id, patent_title, kind, number_of_claims, inventor_name, citation_applicant_count, citation_examiner_count, claim_text, abstract)

# Writing into output file to create json file
json_output_file = open('Group145.json', 'w', encoding='UTF-8')
json_output_file.write(json_data)
json_output_file.close()

<br>

# Section 3. Summary

- #### The task to parse xml data and converting it to csv as well as in json format has given a great insight and lots of learning about the data and its wrangling.
- #### The major task was to identify from where the column data's are fetched
- #### Once the analysis is completed as from where the data is fetched, half the tasks is completed as only retriving part of the file is left. So after a complete analysis of the file, we started to fetch the column data and stored it into lists.
- #### Once all the data's are retrieved, we then converted it into csv file using pandas and json file.
- #### The practical knowledge of this Assignment 1: Parsing Data has made us learn about retrieving and cleaning data and how to represent xml formatted data into a meaningful data such as csv file or json file.